In [1]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



In [2]:
movies_data=pd.read_csv('movies6.csv')
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
movies_data.shape

(4803, 24)

In [28]:
movies_data['title'].tail(50)

4753                              Hayride
4754                        The Naked Ape
4755                             Counting
4756                  The Call of Cthulhu
4757                        Bending Steel
4758                           The Signal
4759                 The Image Revolution
4760                This Is Martin Bonner
4761                         A True Story
4762                    George Washington
4763          Smiling Fish & Goat On Fire
4764            Dawn of the Crescent Moon
4765                       Raymond Did It
4766                       The Last Waltz
4767                       Run, Hide, Die
4768                   The Exploding Girl
4769              The Legend of God's Gun
4770                  Mutual Appreciation
4771    Her Cry: La Llorona Investigation
4772                         Down Terrace
4773                               Clerks
4774                       Pink Narcissus
4775                          Funny Ha Ha
4776                In the Company

In [5]:
movies_data.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [6]:
#movies_data1=movies_data.dropna()

In [7]:
movies_data.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  36
dtype: int64

In [8]:
# selecting the relevant features for recommendation

selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [9]:
# replacing the null valuess with null string

for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna('')

In [10]:
# combining all the 5 selected features

combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [11]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [12]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()

In [13]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [14]:
print(feature_vectors)

  (0, 2431)	0.17272411194153
  (0, 7754)	0.1128035714854756
  (0, 13023)	0.1942362060108871
  (0, 10228)	0.16058685400095302
  (0, 8755)	0.22709015857011816
  (0, 14607)	0.15150672398763912
  (0, 16667)	0.19843263965100372
  (0, 14063)	0.20596090415084142
  (0, 13318)	0.2177470539412484
  (0, 17289)	0.20197912553916567
  (0, 17006)	0.23643326319898797
  (0, 13348)	0.15021264094167086
  (0, 11502)	0.27211310056983656
  (0, 11191)	0.09049319826481456
  (0, 16997)	0.1282126322850579
  (0, 15260)	0.07095833561276566
  (0, 4944)	0.24025852494110758
  (0, 14270)	0.21392179219912877
  (0, 3224)	0.24960162956997736
  (0, 16586)	0.12549432354918996
  (0, 14377)	0.33962752210959823
  (0, 5835)	0.1646750903586285
  (0, 3064)	0.22208377802661425
  (0, 3677)	0.21392179219912877
  (0, 5436)	0.1036413987316636
  :	:
  (4801, 17265)	0.2886098184932947
  (4801, 4834)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6934)	0.2886098184932947
  (4801, 11662)	0.21557500762727902
  (4801, 1671

# Cosine Similarity

In [15]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [16]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [17]:
print(similarity.shape)

(4803, 4803)


# Getting the movie name from the user

In [18]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : iron man


In [19]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = movies_data['title'].tolist()
print (list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [20]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [21]:
close_match = find_close_match[0]
print(close_match)

Iron Man


In [22]:
# finding the index of the movie with title

index_of_the_movie = movies_data.loc[movies_data.title == close_match]['index'].values[0]
print (index_of_the_movie)

68


In [23]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.033569404083678166), (1, 0.054642639091409935), (2, 0.013734950420296663), (3, 0.006468496994386305), (4, 0.03268812370537839), (5, 0.013906699622032292), (6, 0.07692529434993262), (7, 0.23943464854852414), (8, 0.007882072117955854), (9, 0.07598901707288447), (10, 0.07535773020344347), (11, 0.011925591505705335), (12, 0.013707069072870531), (13, 0.01237557919379429), (14, 0.0965674029411586), (15, 0.007285979527733483), (16, 0.22703494343434105), (17, 0.013112402837701539), (18, 0.041403609692602775), (19, 0.07882784318199651), (20, 0.07980853973978243), (21, 0.011266421969523975), (22, 0.006892126774478318), (23, 0.0065988335603164375), (24, 0.01266368471587391), (25, 0.0), (26, 0.21565377247775092), (27, 0.030580057141756805), (28, 0.06452191760630509), (29, 0.014045621630386296), (30, 0.08077023051682258), (31, 0.31465792017331107), (32, 0.028780946249577603), (33, 0.13089286620619242), (34, 0.0), (35, 0.035348674705233986), (36, 0.03185197679774052), (37, 0.00802400546318196

In [24]:
len(similarity_score)

4803

In [25]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_movies)

[(68, 1.0), (79, 0.4089499428792796), (31, 0.31465792017331107), (7, 0.23943464854852414), (16, 0.22703494343434105), (26, 0.21565377247775092), (85, 0.20615037203589281), (182, 0.19573172092755287), (511, 0.16702304899962467), (3623, 0.16107306247442682), (64, 0.1529931129157025), (203, 0.14818074377803042), (174, 0.1471934159351919), (4401, 0.14505390424495612), (101, 0.14401100713769924), (46, 0.14215699425836), (169, 0.13808916985426445), (1740, 0.1362383690866146), (94, 0.13616274148942836), (788, 0.13305362098415385), (126, 0.13263451483519945), (131, 0.1314568163074477), (33, 0.13089286620619242), (2487, 0.1230923886602392), (783, 0.1216705910707248), (138, 0.11845983558699956), (661, 0.11727194663258932), (2442, 0.1172504266287244), (607, 0.11386607377601358), (38, 0.11261375805465461), (2651, 0.11218050255303341), (353, 0.11168017767302062), (122, 0.10849861418377829), (1553, 0.10797337429436599), (1451, 0.10784507500083546), (242, 0.10629913217557008), (618, 0.102542818773457

In [26]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . Avengers: Age of Ultron
5 . The Avengers
6 . Captain America: Civil War
7 . Captain America: The Winter Soldier
8 . Ant-Man
9 . X-Men
10 . Made
11 . X-Men: Apocalypse
12 . X2
13 . The Incredible Hulk
14 . The Helix... Loaded
15 . X-Men: First Class
16 . X-Men: Days of Future Past
17 . Captain America: The First Avenger
18 . Kick-Ass 2
19 . Guardians of the Galaxy
20 . Deadpool
21 . Thor: The Dark World
22 . G-Force
23 . X-Men: The Last Stand
24 . Duets
25 . Mortdecai
26 . The Last Airbender
27 . Zathura: A Space Adventure
28 . Southland Tales
29 . Sky Captain and the World of Tomorrow


In [32]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : Titenic
Movies suggested for you : 

1 . Titanic
2 . The Dilemma
3 . The Day the Earth Stood Still
4 . Revolutionary Road
5 . Cheri
6 . Heavenly Creatures
7 . Plush
8 . Primary Colors
9 . The Bridge of San Luis Rey
10 . Saving Silverman
11 . Gangs of New York
12 . Flushed Away
13 . Sense and Sensibility
14 . Stuck on You
15 . The Great Gatsby
16 . Almost Famous
17 . Shutter Island
18 . The Brothers Bloom
19 . Contagion
20 . The Blind Side
21 . Little Black Book
22 . This Is the End
23 . The Host
24 . Body of Lies
25 . Romance & Cigarettes
26 . Ladyhawke
27 . The Aviator
28 . The Believer
29 . The Phantom
